# Pipeline di Preparazione Dati per MSSegSAM

Questo notebook permette la preparazione dei dati per il training del modello MSSegSAM.

La pipeline è suddivisa in due fasi distinte e sequenziali:

1.  **Fase 1: Preprocessing e Standardizzazione (MRI $\rightarrow$ NIfTI MNI152)**
    *   Trasformazione dei dati grezzi disomogenei in un formato standardizzato.
    *   Applicazione di Skull Stripping, Registrazione allo spazio MNI e Correzione del Bias Field.

2.  **Fase 2: Conversione in Formato COCO (NIfTI $\rightarrow$ COCO JSON)**
    *   Estrazione delle slice 2D dai volumi 3D.
    *   Generazione delle annotazioni in formato standard MS-COCO per il training.

## Setup dell'Ambiente

In [ ]:
import os
import sys
from pathlib import Path

current_dir = Path(os.getcwd())
project_root = current_dir.parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

try:
    import src.pipeline as pipeline_module
    import src.coco_converter as coco_module
    print("Moduli caricati correttamente.")
except ImportError as e:
    print(f"Errore nell'importazione dei moduli: {e}")

## Fase 1: Preprocessing

In [ ]:
INPUT_RAW_DIR = "../Datasets_raw"
OUTPUT_PROC_DIR = "../Datasets_Process"

if __name__ == "__main__":
    try:
        pipeline_module.run_dataset_pipeline(INPUT_RAW_DIR, OUTPUT_PROC_DIR)
    except Exception as e:
        print(f"\nErrore durante il preprocessing: {e}")

### Suddivisione Train/Val/Test

Completata la Fase 1, i dati sono organizzati per Dataset -> Paziente -> Timepoint.

Prima della conversione COCO, è consigliato organizzare i dati in sottocartelle `train`, `val` e `test` all'interno di ogni dataset.

La fase successiva di conversione COCO rileva automaticamente queste cartelle. Se i dati sono già divisi, il converter genererà tre file JSON separati (`train/annotations.json`, `val/...`, `test/...`), pronti per il training del modello. Se i dati non sono divisi, verrà generato un unico dataset "flat".


## Fase 2: Conversione in Formato COCO

| Parametro | Tipo | Descrizione | Default |
| :--- | :--- | :--- | :--- |
| **`input_dir`** | `str` | Root directory containing **Processed** datasets. | **Required** |
| **`output_dir`** | `str` | Target directory for the generated COCO dataset. | `dataset_COCO` |
| **`dataset_ids`** | `list` | Indices of sub-datasets to process. `["0", "2"]` OR `["all"]`. | `["all"]` |
| **`slice_range`** | `list` | Definisce quali slice assiali estrarre dal volume 3D. `["0", "181"]` OR `["all"]`. | `["all"]` |
| **`remove_empty`** | `bool` | If True, skips slices with no Ground Truth lesions. | `False` |
| **`all_timepoints`** | `bool` | If True, process all timepoints instead of just the last one. | `False` |
| **`modality`** | `str` | MRI modality to extract (T1, T2, FLAIR). | `"FLAIR"` |

In [ ]:
INPUT_NIFTI_DIR = "../Datasets_Process"
OUTPUT_COCO_DIR = "../Datasets_COCO"

# Config
DATASET_IDS = ["all"]
SLICE_RANGE = ["all"]
MODALITY = "FLAIR"
REMOVE_EMPTY = False
ALL_TIMEPOINTS = False

if __name__ == "__main__":
    converter = coco_module.COCOConverter(
        input_dir=INPUT_NIFTI_DIR,
        output_dir=OUTPUT_COCO_DIR,
        dataset_ids=DATASET_IDS,
        slice_range=SLICE_RANGE,
        modality=MODALITY,
        remove_empty=REMOVE_EMPTY,
        all_timepoints=ALL_TIMEPOINTS
    )
    
    try:
        converter.run()
    except Exception as e:
        print(f"\nErrore durante la conversione COCO: {e}")